In [ ]:
#!pip install lxml
#!pip install openai

In [2]:
import json
from pprint import pprint
import os
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from lxml import etree
import json
import openai
import os
# Constants
API_SOURCE = "mediastack"
current_date, time = str(datetime.today()).split(" ")

path = os.path.join("News storage", API_SOURCE, current_date)

file_name = current_date + "_" + API_SOURCE + "_" + "extracted_news.json"
file_path = os.path.join(path, file_name)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
# OpenAI profile setup
openai.organization = "org-ha7l6qsycbULfVHdz5MwgKgr"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
regex_title = re.compile("title|headline|titular|titulo")
regex_body = re.compile("articleBody")
regex_date_published = re.compile("date.*[pP]ub.*")
regex_date_modified = re.compile("date.*[mM]od.*")
regex_tags = re.compile("tag[s]?|topic[s]?|tema[s]?|etiqueta[s]?|keyword[s]?")

def find_keys(elements_json):
    news_data = {}
    title_found = False
    body_found = False
    date_pub_found = False
    date_mod_found = False
    tag_found = False
    news_data["title"] = None
    news_data["body"] = None
    news_data["creation_date"] = None
    news_data["modification_date"] = None
    news_data["tags"] = None
    
    for element_json in elements_json:
        json_file = json.loads(element_json.get_text(), 
                               strict=False)
        if isinstance(json_file, list):
            json_file = json_file[0]
        for key in json_file:
            if not title_found and regex_title.search(key):
                news_data["title"] = json_file[key]
                title_found = True
            elif not body_found and regex_body.search(key):
                news_data["body"] = remove_tags(json_file[key])
                body_found = True
            elif not date_pub_found and regex_date_published.search(key):
                news_data["creation_date"] = json_file[key]
                date_pub_found = True
            elif not date_mod_found and regex_date_modified.search(key):
                news_data["modification_date"] = json_file[key]
                date_mod_found = True
            elif not tag_found and regex_tags.search(key):
                news_data["tags"] = json_file[key]
                data["tags"] = ";".join(data["tags"])
                tag_found = True
            if all([title_found, body_found, date_pub_found, date_mod_found, tag_found]):
                break
    return news_data
def remove_tags(text):
    return re.sub("<.*?>", "", text)

## Extracion of full article from given url

### el mundo

In [ ]:
url = "https://elmundo.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

In [ ]:
# With headers
response = requests.get(url, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

### El Español

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
main_root = "https://www.elespanol.com"

# With headers
response = requests.get(main_root, headers=headers)
print("Response code:", response.status_code)  # HTTP status code
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

#sections = parsed_hmtl.find_all("li", {"itemprop":"name"})
#urls_sections = [x.find("a").attrs["href"] for x in sections]
#print(urls_sections)

a_tags = parsed_hmtl.find_all("a", href=re.compile("/[0-9]+/"))

valid_news_urls = []
section_urls = []
for a in a_tags:
    try:
        href = a["href"]
        print(href)
        if not href.startswith(main_root):
            href = main_root + href
        if href.endswith("//"):
            section_urls.append(href)
        valid_news_urls.append(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

### 20minutos

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.20minutos.es/"
main_root = "https://www.20minutos.es/"

date = "20230524"
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

#sections = parsed_hmtl.find_all("li", {"itemprop":"name"})
#urls_sections = [x.find("a").attrs["href"] for x in sections]
#print(urls_sections)

a_tags = parsed_hmtl.find_all("a", href=re.compile("/[0-9]+/"))

valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        print(href)
        if not href.startswith(main_root):
            href = main_root + href
        valid_news_urls.append(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"]for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)
len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

### larazon

In [ ]:
url = "https://www.larazon.es"
main_root = "https://www.larazon.es"

date = "20230524"
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

sections = parsed_hmtl.find("ul", attrs={"class": "custom_carousel_container"})
urls_sections = [x.attrs["href"] for x in sections.find_all("a")]

a_tags = parsed_hmtl.find_all("a", href=re.compile(".html$"))

valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if not href.startswith(main_root):
            continue
        valid_news_urls.append(href)
        print(href)
    except:
        not_valid_news_urls.append(href)
len(valid_news_urls), len(not_valid_news_urls)

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

### La vanguardia

In [35]:
def find_attr_article(tag):
    return tag.has_attr('class') and "article" in " ".join(tag.attrs["class"])
def find_attr_article(tag):
    return tag.name == "article" and tag.attrs["href"]
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com"
main_root = "https://www.lavanguardia.com"
date = current_date.replace("-", "")
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
#articles = parsed_hmtl.find_all(find_attr_article)
#raw_urls = [article.find("a").attrs["href"] for article in articles]
articles = parsed_hmtl.find_all("a", href=re.compile("/[0-9]{8}/[0-9]{1,}/"))
raw_urls = [article.attrs["href"] for article in articles]
valid_news_urls = []
rejected_urls = []
for raw_url in raw_urls:
    if not re.search(r"/[0-9]{8}/[0-9]{1,}/", raw_url):
        rejected_urls.append(raw_url)
        continue
    if raw_url.startswith("/"):
        raw_url = main_root + raw_url
    elif main_root not in raw_url:
        rejected_urls.append(raw_url)
        continue
    valid_news_urls.append(raw_url)
    print(raw_url)
valid_news_urls = list(set(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

Response code: 200
https://www.lavanguardia.com/television/20230628/9074951/feijoo-revela-que-mas-le-preocupa-julio-iglesias-espana-esta-obsesionado.html
https://www.lavanguardia.com/internacional/20230628/9074752/llegan-canada-restos-sumergible-titan-recuperados-atlantico.html
https://www.lavanguardia.com/gente/20230628/9072453/muere-carmen-sevilla-ultima-hora-reacciones-curiosidades-en-directo-hoy.html?_gl=1*1unsjb2*_gcl_au*NDIzNzk3MzczLjE2ODUyNzcxOTM.
https://www.lavanguardia.com/television/20230628/9074885/soltero-regresa-first-dates-haber-sido-plantado-anterior-cita-segunda-vencida-mmn.html
https://www.lavanguardia.com/gente/20230628/9072627/sorpresa-atestado-policial-accidente-sergio-rico-haber-testigos-directos-mmn.html
https://www.lavanguardia.com/cultura/20230628/9074917/madonna-suspende-gira-haber-ingresada-uci.html
https://www.lavanguardia.com/loterias/20230628/9058839/once-resultado-sorteo-cupon-diario-hoy-miercoles-28-junio-2023-ctx.html
https://www.lavanguardia.com/loteri

## Okdiario

In [ ]:
def find_attr_article(tag):
    return tag.name == "article" and tag.attrs["href"]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

url = "https://okdiario.com"
main_root = "https://okdiario.com"
date = current_date.replace("-", "")
# With headers
response = requests.get(url, headers=headers)
print("Response code:", response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
#articles = parsed_hmtl.find_all(find_attr_article)
articles = parsed_hmtl.find_all("a", href=re.compile("-[0-9]+$"))
#raw_urls = [article.find("a").attrs["href"] for article in articles]
raw_urls = [article.attrs["href"] for article in articles]
valid_news_urls = []
rejected_urls = []
for raw_url in raw_urls:
    if not re.search(r"-[0-9]+$", raw_url):
        rejected_urls.append(raw_url)
        continue
    if raw_url.startswith("/"):
        raw_url = main_root + raw_url
    elif main_root not in raw_url:
        rejected_urls.append(raw_url)
        continue
    valid_news_urls.append(raw_url)
    print(raw_url)
valid_news_urls = list(set(valid_news_urls))
len(valid_news_urls), len(rejected_urls)

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

## ABC

In [ ]:
url = "https://abc.es"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
main_root = "https://www.abc.es"
# With headers
response = requests.get(url, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

a_tags = parsed_hmtl.find_all("a", href=re.compile("-[a-z]{1,2}.html$"))
valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if main_root not in href:
            href = main_root + href
        valid_news_urls.append(href)
        print(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if data["tags"] is None:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

### Vandal

In [ ]:
url = "https://vandal.elespanol.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
main_root = "https://vandal.elespanol.com"
# With headers
response = requests.get(url, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

a_tags = parsed_hmtl.find_all("a", href=re.compile("/[0-9]+/"))
valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if main_root not in href:
            href = main_root + href
        valid_news_urls.append(href)
        print(href)
    except:
        continue
print(len(valid_news_urls))

news_media_data = []
not_processed_urls = []
valid_news_urls = list(set(valid_news_urls))
for i, news_url in enumerate(valid_news_urls):
    resp_url_news = requests.get(news_url, 
                                 headers=headers)
    parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                     "html.parser")
    elements_json = parsed_news_hmtl.find_all("script", 
                                              attrs={"type": re.compile("json$")})
    data = find_keys(elements_json)
    data["url"] = news_url
    if not data["tags"]:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"property": "article:tag"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    if not data["tags"]:
        elements_with_tags = parsed_news_hmtl.find_all("meta", 
                                                       attrs={"name": "keywords"})
        data["tags"] = [x.attrs["content"] for x in elements_with_tags]
        data["tags"] = ";".join(data["tags"])
    news_media_data.append(data)

len(news_media_data), \
news_media_data[0].keys(), \
all([len(x) == 6 for x in news_media_data]), \
all([x["tags"] is not None for x in news_media_data])

In [ ]:
news_media_data

## Extract Spain media url

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
main_root = "https://www.prensaescrita.com/prensadigital.php"
# With headers
response = requests.get(main_root, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

#a_tags = parsed_hmtl.find_all("a", href=re.compile("www"))

partial_text = "www."
links = parsed_hmtl.find_all("a", string=lambda text: text and partial_text in text)

200


In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
y, m = str(datetime.today()).split("-")[:2]
main_root = "https://www.abc.es"
main_root = "https://theobjective.com/"
main_root = "https://www.elespanol.com"
main_root = "https://vandal.elespanol.com"
main_roots = [x.attrs["href"] for x in links]
# With headers
response = requests.get(main_root, headers=headers)
print(response.status_code)  # HTTP status code

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

a_tags = parsed_hmtl.find_all("a", 
                              href=re.compile(f"{y}[^0-9]?{m}[^0-9]?[0-9]{{1,2}}")) + \
         parsed_hmtl.find_all("a", 
                              href=re.compile(r"/[0-9]{5,}/")) + \
         parsed_hmtl.find_all("a", 
                              href=re.compile(r".html$"))
a_tags = pd.Series(a_tags).unique().tolist()
a_tags
valid_news_urls = []
not_valid_news_urls = []
for a in a_tags:
    try:
        href = a["href"]
        if href.startswith("/"):
            href = main_root + href
        if main_root in href:
            valid_news_urls.append(href)
            print(href)
    except:
        continue
print(len(valid_news_urls))

200
https://vandal.elespanol.com/noticia/1350763526/filtrados-los-juegos-de-ps-plus-de-julio-alan-wake-remastered-cod-bo-cold-war-y-endling/
https://vandal.elespanol.com/noticia/1350763530/ps5-costara-100-euros-menos-por-tiempo-limitado-ofertas-adicionales-en-accesorios-y-juegos/
https://vandal.elespanol.com/avances/switch/super-mario-rpg-remake/143853/1
https://vandal.elespanol.com/avances/pc/lysfanga-the-time-shift-warrior/143016/1
https://vandal.elespanol.com/avances/pc/ride-5/133942/1
https://vandal.elespanol.com/noticia/1350763555/un-documento-mal-redactado-de-sony-revela-cuanta-gente-juega-a-call-of-duty-en-playstation/
https://vandal.elespanol.com/noticia/1350763557/efootball-2024-el-juego-gratuito-de-futbol-de-konami-llegara-entre-agosto-y-septiembre/
https://vandal.elespanol.com/noticia/1350763559/baldurs-gate-3-se-adelanta-al-3-de-agosto-en-pc-y-se-retrasa-al-9-de-septiembre-en-ps5/
https://vandal.elespanol.com/noticia/1350763534/sony-desvela-sin-querer-cuanto-costo-desarroll